In [39]:
!pip install tweet-preprocessor

In [40]:
import pandas as pd
import matplotlib.pyplot as polt
from textblob import TextBlob
from datetime import datetime, timedelta
import requests
import pandas as pd
import preprocessor as p

In [41]:
# read bearer token for authentication
with open('bearer.txt') as fp:
    BEARER_TOKEN = fp.read()

ticker = 'GME'
company = 'Gamestop'
# setup the API request
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': '('+ ticker+') (lang:en)',
    'max_results': '100',
    'tweet.fields': 'created_at,lang'
}

dtformat = '%Y-%m-%dT%H:%M:%SZ'  # the date format string required by twitter

# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

def get_data(tweet):
    data = {
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'text': tweet['text']
    }
    return data
    
now = datetime.now() - timedelta(seconds=20) # get the current datetime
last_week = now - timedelta(days=1)  # datetime one day ago = the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API

df = pd.DataFrame()  # initialize dataframe to store tweets
while True:
    if datetime.strptime(now, dtformat) < last_week:
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    params['start_time'] = pre60
    params['end_time'] = now
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    for tweet in response.json()['data']:
        row = get_data(tweet)
        df = df.append(row, ignore_index=True)

In [42]:
df.head()

,created_at,id,text
0,2021-04-13T03:31:00.000Z,1381812133495717889,RT @equijohn: $BRSE new streaming Partner \nht...
1,2021-04-13T03:30:43.000Z,1381812065409589250,RT @weaponizedgamma: LIKE IF YOU ARE NOT FUCKI...
2,2021-04-13T03:30:33.000Z,1381812020815622150,@thedesertwolfe $gme six degrees of separation.
3,2021-04-13T03:30:22.000Z,1381811975609446404,@HesterPeirce Why don’t you guys get off your ...
4,2021-04-13T03:30:22.000Z,1381811974540042240,RT @NEVERBOW: @RobinhoodApp when will $coin be...


In [43]:
filename = company + ".csv"
df.to_csv(filename,index = False)

In [70]:
df = pd.read_csv(filename)

In [89]:
sentiment = []
scores = []
for tweet in df.text:
    tweet = p.clean(tweet)
    analysis = TextBlob(tweet)
    score = analysis.sentiment.polarity 
    scores.append(score)
    if score > 0:
        structured_tweet['sentiment'] = 'positive'
        sentiment.append("positive")
    elif score == 0:
        sentiment.append("neutral")
    else:
        sentiment.append("negative")
df["sentiment"] = sentiment
df["score"] = scores

In [90]:
pos = 100 * len(df[df['sentiment'].str.contains('positive')]) / len(df)
neg = 100 * len(df[df['sentiment'].str.contains('negative')]) / len(df)
neu = 100 * len(df[df['sentiment'].str.contains('neutral')]) / len(df)
print("Positive tweet {} %".format(pos))
print("Negative tweets {} %".format(neg))
print("Neutral tweets {} % ".format(neu))

Positive tweet 42.46861924686193 %
Negative tweets 16.359832635983263 %
Neutral tweets 41.17154811715481 % 


In [78]:
df = df.sort_values(by=['score'])

In [79]:
dis = df.filter(["text","score"], axis=1)

In [83]:
dis.iloc[:5]

,text,score
1202,YOU HEAR THAT HEDGIES??? WE ARE FUCKING MASOCH...,-1.00000
481,RT @TalatiTapan: @ChrisRandone I dont know but...,-1.00000
1077,Lets fucking goooo!!!!! #GameStop $GME,-1.00000
618,RT @madaznfootballr: Trader 'madazmoney' trade...,-0.80000
67,"RT @PappaApe: 😳 Game Over,Done, Out Of Here, s...",-0.78125
